In [51]:
import pyodbc
import yaml
from sqlalchemy import create_engine
import urllib
import pandas as pd

## Connection

In [52]:
# Load api-keys
config_stream = open("../config.yml",'r')
config = yaml.load(config_stream, Loader=yaml.BaseLoader)
sql_db_config = config['azure_sql_db']

server = 'ffa-server-1.database.windows.net'
database = 'External_API_Data'
username = sql_db_config['username']
password = sql_db_config['password']
driver = '{ODBC Driver 17 for SQL Server}'

connection_string = 'DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password
#'DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)

In [54]:
cnxn = pyodbc.connect(connection_string)
cursor = cnxn.cursor()
#Sample select query
cursor.execute("SELECT @@version;") 
row = cursor.fetchone() 
while row: 
    print(row[0])
    row = cursor.fetchone()
cnxn.commit()
cursor.close()

Microsoft SQL Azure (RTM) - 12.0.2000.8 
	May 12 2022 23:11:24 
	Copyright (C) 2022 Microsoft Corporation



## From Pandas to SQL by defining table first

In [9]:
cnxn = pyodbc.connect(connection_string)
cursor = cnxn.cursor()
df_test = pd.read_csv('./data_checks/HR_test.csv')
df_test = df_test[['DepartmentID','Name','GroupName']]
for index, row in df_test.iterrows():
     cursor.execute("INSERT INTO HumanResources.Department (DepartmentID,Name,GroupName) values(?,?,?)", row.DepartmentID, row.Name, row.GroupName)
cnxn.commit()
cursor.close()

## From SQL to Pandas (Loading)

In [55]:
# Read from SQL database to pandasd dataframe
cnxn = pyodbc.connect(connection_string)
id = 11
query = f"SELECT * FROM HumanResources.Department WHERE DepartmentID < {id}"
df = pd.read_sql_query(query, con=cnxn)
cnxn.commit()

In [56]:
df

,DepartmentID,Name,GroupName
0,1,Engineering,Research and Development
1,2,Tool Design,Research and Development
2,3,Sales,Sales and Marketing
3,4,Marketing,Sales and Marketing
4,5,Purchasing,Inventory Management
5,6,Research and Development,Research and Development
6,7,Production,Manufacturing
7,8,Production Control,Manufacturing
8,9,Human Resources,Executive General and Administration
9,10,Finance,Executive General and Administration


## From Pnadas to SQL-DB (Auto-table creation)

In [45]:
df_test_2 = df_test.copy()
df_test_2['DepartmentID'] = df_test_2['DepartmentID']+30 
df_test_2.head()

,DepartmentID,Name,GroupName
0,31,Engineering,Research and Development
1,32,Tool Design,Research and Development
2,33,Sales,Sales and Marketing
3,34,Marketing,Sales and Marketing
4,35,Purchasing,Inventory Management


In [47]:
cnxn = pyodbc.connect(connection_string)
params = urllib.parse.quote_plus(connection_string)
# define sql alchemy engine
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
df_test_2.to_sql(name="Department",schema="HumanResources", con=engine, index = False, if_exists='append')
cnxn.commit()

